In [3]:
import pandas as pd
import numpy as np

In [4]:
df_2023 = pd.read_csv('../data/raw_data/uk_2023.csv', encoding='iso-8859-2', delimiter=';')
df_2022 = pd.read_csv('../data/raw_data/uk_2022.csv', encoding='iso-8859-2', delimiter=';')
df_2021 = pd.read_csv('../data/raw_data/uk_2021.csv', encoding='iso-8859-2', delimiter=';')


In [5]:
result = pd.concat([df_2023, df_2022, df_2021], axis=0, ignore_index=True)

In [6]:
result['colour_rating_new'] = result['colour_rating'].combine_first(result['colour_rating.1'])


In [7]:
result = result.drop(columns=['colour_rating', 'colour_rating.1'])

In [8]:
result['total_baseline'] = result['total_baseline'].str.replace('Ł', '', regex=False)

In [9]:
result['total_baseline'] = pd.to_numeric(result['total_baseline'], errors='coerce')

In [10]:
result['start_date'] = pd.to_datetime(result['start_date'], errors='coerce')

C:\Users\fraus\AppData\Local\Temp\ipykernel_32740\2580846696.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  result['start_date'] = pd.to_datetime(result['start_date'], errors='coerce')


In [11]:
result['end_date'] = pd.to_datetime(result['end_date'], errors='coerce')

C:\Users\fraus\AppData\Local\Temp\ipykernel_32740\628838394.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  result['end_date'] = pd.to_datetime(result['end_date'], errors='coerce')


In [12]:
result.columns

Index(['year', 'project_number', 'project_name', 'department',
       'report_category', 'description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'total_baseline', 'forecast_incl_NGC',
       'variance_comment', 'wlc_baseline_incl_NCG', 'budget_comment',
       'TOTAL Baseline Benefits (Łm)', 'benefits_comment', 'yearly_forecast',
       'colour_rating_new'],
      dtype='object')

In [13]:
result['forecast_incl_NGC'] = result['forecast_incl_NGC'].str.replace('Ł', '', regex=False)
result['forecast_incl_NGC'] = pd.to_numeric(result['forecast_incl_NGC'], errors='coerce')

In [14]:
result['wlc_baseline_incl_NCG'] = result['wlc_baseline_incl_NCG'].str.replace('Ł', '', regex=False)
result['wlc_baseline_incl_NCG'] = pd.to_numeric(result['wlc_baseline_incl_NCG'], errors='coerce')

In [15]:
result['TOTAL Baseline Benefits (Łm)'] = result['TOTAL Baseline Benefits (Łm)'].str.replace('Ł', '', regex=False)
result['TOTAL Baseline Benefits (Łm)'] = pd.to_numeric(result['TOTAL Baseline Benefits (Łm)'], errors='coerce')

In [16]:
result['yearly_forecast'] = result['yearly_forecast'].str.replace('Ł', '', regex=False)
result['yearly_forecast'] = pd.to_numeric(result['yearly_forecast'], errors='coerce')

In [17]:
allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
result['colour_rating_new'] = result['colour_rating_new'].apply(lambda x: x if x in allowed_values else np.nan)

In [18]:
result['colour_rating_new'].unique()

array(['Amber', 'Green', 'Red', nan, 'Amber/red', 'Amber/Green'],
      dtype=object)

In [19]:
result = result.rename(columns={'colour_rating_new': 'colour_rating'})

In [20]:
result['yearly_budget'] = np.nan

In [22]:
result['yearly_forecast_new'] = result['yearly_forecast'].combine_first(result['forecast_incl_NGC'])

In [24]:
result.columns

Index(['year', 'project_number', 'project_name', 'department',
       'report_category', 'description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'total_baseline', 'forecast_incl_NGC',
       'variance_comment', 'wlc_baseline_incl_NCG', 'budget_comment',
       'TOTAL Baseline Benefits (Łm)', 'benefits_comment', 'yearly_forecast',
       'colour_rating', 'yearly_budget', 'yearly_forecast_new'],
      dtype='object')

In [27]:
result = result.drop(columns=['yearly_forecast', 'forecast_incl_NGC'])

In [28]:
result = result.rename(columns={'yearly_forecast_new': 'yearly_forecast'})

In [35]:
result.columns

Index(['year', 'project_number', 'project_name', 'department',
       'report_category', 'description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'total_baseline', 'variance_comment',
       'wlc_baseline_incl_NCG', 'budget_comment',
       'TOTAL Baseline Benefits (Łm)', 'benefits_comment', 'colour_rating',
       'yearly_budget', 'yearly_forecast'],
      dtype='object')

In [37]:
result = result.drop(columns=['yearly_budget'])

In [38]:
result = result.rename(columns={'total_baseline': 'yearly_budget'})

In [39]:
column_order = ['project_name','department','colour_rating','description_aims','rating_comment','start_date','end_date','schedule_comment','yearly_budget','yearly_forecast','wlc_baseline_incl_NCG','variance_comment','budget_comment','year','report_category','project_number','TOTAL Baseline Benefits (Łm)','benefits_comment']

In [41]:
result = result[column_order]

In [43]:
result.to_csv('../data/raw_data/2021_2023.csv')

In [49]:
df_rauschgold = pd.read_csv('../data/raw_data/2021_2023.csv')
df_emna = pd.read_csv('df_2014_2017.csv')

In [50]:
result = pd.concat([df_rauschgold, df_emna], axis=0, ignore_index=True)

In [52]:
result.columns

Index(['Unnamed: 0', 'project_name', 'department', 'colour_rating',
       'description_aims', 'rating_comment', 'start_date', 'end_date',
       'schedule_comment', 'yearly_budget', 'yearly_forecast',
       'wlc_baseline_incl_NCG', 'variance_comment', 'budget_comment', 'year',
       'report_category', 'project_number', 'TOTAL Baseline Benefits (Łm)',
       'benefits_comment'],
      dtype='object')

In [54]:
result = result.drop(columns=['Unnamed: 0'])

In [56]:
result.to_csv('emna_franzi.csv')

In [57]:
df_2018 = pd.read_csv("../data/raw_data/uk_2018.csv", encoding="latin2")
df_2019 = pd.read_csv("../data/raw_data/uk_2019.csv", encoding="latin2")
df_2020 = pd.read_csv("../data/raw_data/uk_2020.csv", encoding="latin2")
df = pd.concat([df_2020, df_2019, df_2018])
df.to_csv("../data/raw_data/uk_2018-2020.csv")

In [58]:
df1 = pd.read_csv('../data/raw_data/emna_franzi.csv')

In [59]:
result = pd.concat([df1, df], axis=0, ignore_index=True)

In [61]:
result = result.drop(columns=['Unnamed: 0'])

In [63]:
result.to_csv('dataset_EDA.csv')